# Подключаем библиотеки

In [ ]:
import xgboost as xgb
import lightgbm as lgb
import pandas as pd
import numpy as np
from geopy.distance import geodesic 
import math
from sklearn.metrics import mean_absolute_error, r2_score, median_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
import warnings
warnings.filterwarnings('ignore')

# Объявляем функции

In [ ]:
#Вычисляет азимут по двум заданным точкам, код взят отсюда: https://pastebin.com/PHeWmiEN
def get_azimuth(latitude, longitude):
 
    rad = 6372795

    llat1 = city_center_coordinates[0]
    llong1 = city_center_coordinates[1]
    llat2 = latitude
    llong2 = longitude

    lat1 = llat1*math.pi/180.
    lat2 = llat2*math.pi/180.
    long1 = llong1*math.pi/180.
    long2 = llong2*math.pi/180.

    cl1 = math.cos(lat1)
    cl2 = math.cos(lat2)
    sl1 = math.sin(lat1)
    sl2 = math.sin(lat2)
    delta = long2 - long1
    cdelta = math.cos(delta)
    sdelta = math.sin(delta)

    y = math.sqrt(math.pow(cl2*sdelta,2)+math.pow(cl1*sl2-sl1*cl2*cdelta,2))
    x = sl1*sl2+cl1*cl2*cdelta
    ad = math.atan2(y,x)

    x = (cl1*sl2) - (sl1*cl2*cdelta)
    y = sdelta*cl2
    z = math.degrees(math.atan(-y/x))

    if (x < 0):
        z = z+180.

    z2 = (z+180.) % 360. - 180.
    z2 = - math.radians(z2)
    anglerad2 = z2 - ((2*math.pi)*math.floor((z2/(2*math.pi))) )
    angledeg = (anglerad2*180.)/math.pi
    
    return round(angledeg, 2)

#Вычисляет среднюю абсолютную процентную ошибку
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#Вычисляет медианную абсолютную процентную ошибку
def median_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.median(np.abs((y_true - y_pred) / y_true)) * 100

#Печатает рассчитанные значения коэффициента детерминации и ошибок
def print_metrics(prediction, val_y):
    val_mae = mean_absolute_error(val_y, prediction)
    median_AE = median_absolute_error(val_y, prediction)
    r2 = r2_score(val_y, prediction)

    print('')
    print('R\u00b2: {:.2}'.format(r2))
    print('')
    print('Средняя абсолютная ошибка: {:.3} %'.format(mean_absolute_percentage_error(val_y, prediction)))
    print('Медианная абсолютная ошибка: {:.3} %'.format(median_absolute_percentage_error(val_y, prediction)))

# Загружаем датасет и делаем первичную обработку

In [ ]:
#При помощи библиотеки pandas считываем csv-файл и преобразуем его в формат датафрейма (таблицы)
file_path = 'https://raw.githubusercontent.com/maxbobkov/ml_moscow_flats/master/moscow_dataset.csv'
df = pd.read_csv(file_path)

#Выводим 5 первых строк датафрейма
df.head(5)

In [ ]:
#Создаем новый столбец Стоимость 1 кв.м путем построчного деления стоимостей квартир на их общие площади
df['priceMetr'] = df['price']/df['totalArea']

#Задаем широту и долготу центра города и рассчитываем для каждой квартиры расстояние от центра и азимут 
city_center_coordinates = [55.7522, 37.6156]
df['distance'] = list(map(lambda x, y: geodesic(city_center_coordinates, [x, y]).meters, df['latitude'], df['longitude']))
df['azimuth'] = list(map(lambda x, y: get_azimuth(x, y), df['latitude'], df['longitude']))

#Выбираем из датафрейма только те квартиры, которые расположены не дальше 40 км от центра города и объявления о продаже которых были созданы не ранее 1 июня 2019 г. 
df = df.loc[(df['distance'] < 40000) & (df['creationDate'] > '2019-06-01')]

#Выводим сводную информацию о датафрейме и его столбцах (признаках)
df.info()

# Удаляем выбросы

In [ ]:
#Вычисляем строки со значениями-выбросами 
first_quartile = df.quantile(q=0.25)
third_quartile = df.quantile(q=0.75)
IQR = third_quartile - first_quartile
outliers = df[(df > (third_quartile + 1.5 * IQR)) | (df < (first_quartile - 1.5 * IQR))].count(axis=1)
outliers.sort_values(axis=0, ascending=False, inplace=True)

#Удаляем из датафрейма 500 строк, подходящих под критерии выбросов
outliers = outliers.head(500)
df.drop(outliers.index, inplace=True)

# Превращаем категорийные признаки в числовые

In [ ]:
#Вычисляем столбцы с категорийными признаками, затем заменяем их на числа
categorical_columns = df.columns[df.dtypes == 'object']
labelencoder = LabelEncoder()
for column in categorical_columns:
    df[column] = labelencoder.fit_transform(df[column])

#Выводим сводную информацию о датафрейме и его столбцах (признаках), чтобы убедиться, что теперь они все содержат цифровые значения
df.info()

# Создаем целевую переменную, делим датасет на выборки

In [ ]:
#Назначаем целевой переменной цену 1 кв. метра, а можно и цену всей квартиры, тогда будет y = df['price']
y = df['priceMetr']

#Создаем список признаков, на основании которых будем строить модели
features = [
            'wallsMaterial', 
            'floorNumber', 
            'floorsTotal', 
            'totalArea', 
            'kitchenArea',
            'distance',
            'azimuth'
           ]

#Создаем датафрейм, состоящий из признаков, выбранных ранее
X = df[features]

#Проводим случайное разбиение данных на выборки для обучения (train) и валидации (val), по умолчанию в пропорции 0.75/0.25
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# Модель Random forest

In [ ]:
#Создаем регрессионную модель случайного леса 
rf_model = RandomForestRegressor(n_estimators=500, 
                                 n_jobs=-1,  
                                 bootstrap=False,
                                 criterion='mse',
                                 max_features=6,
                                 random_state=1)

#Проводим подгонку модели на обучающей выборке 
rf_model.fit(train_X, train_y)

#Вычисляем предсказанные значения цен на основе валидационной выборки
rf_prediction = rf_model.predict(val_X)

#Вычисляем и печатаем величины ошибок при сравнении известных цен квартир из валидационной выборки с предсказанными моделью
print_metrics(rf_prediction, val_y)

# Модель XGBoost

In [ ]:
#Создаем регрессионную модель XGBoost
xgb_model = xgb.XGBRegressor(objective ='reg:gamma', 
                             learning_rate = 0.05,
                             max_depth = 12, 
                             n_estimators = 3000,
                             nthread = 8,
                             eval_metric = 'gamma-nloglik', 
                             )

#Проводим подгонку модели на обучающей выборке 
xgb_model.fit(train_X, train_y, )

#Вычисляем предсказанные значения цен на основе валидационной выборки
xgb_prediction = xgb_model.predict(val_X)

#Вычисляем и печатаем величины ошибок при сравнении известных цен квартир из валидационной выборки с предсказанными моделью
print_metrics(xgb_prediction, val_y)

# Модель LightGBM

In [ ]:
#Создаем регрессионную модель LightGBM
lgb_model = lgb.LGBMRegressor(
                              objective='regression',
                              num_leaves=200,
                              learning_rate=0.05, 
                              n_estimators=2000,
                              max_bin = 300, 
                              bagging_fraction = 0.9,
                              bagging_freq = 70, 
                              feature_fraction_seed=5, 
                              bagging_seed=2,
                              min_data_in_leaf = 2, 
                              boosting_type= 'gbdt',
                             )

#Проводим подгонку модели на обучающей выборке 
lgb_model.fit(train_X, train_y)

#Вычисляем предсказанные значения цен на основе валидационной выборки
lgb_prediction = lgb_model.predict(val_X)

#Вычисляем и печатаем величины ошибок при сравнении известных цен квартир из валидационной выборки с предсказанными моделью
print_metrics(lgb_prediction, val_y)

# Усреднение предсказаний моделей

In [ ]:
#Усредняем предсказания всех трех моделей
prediction = rf_prediction * 0.33 + xgb_prediction * 0.34 + lgb_prediction * 0.33

#Вычисляем и печатаем величины ошибок для усредненного предсказания
print_metrics(prediction, val_y)

# Изучаем важность признаков в модели Random forest

In [ ]:
#Рассчитываем важность признаков в модели Random forest
importances = rf_model.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf_model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

#Печатаем рейтинг признаков
print("Рейтинг важности признаков:")
for f in range(X.shape[1]):
    print("%d. %s (%f)" % (f + 1, features[indices[f]], importances[indices[f]]))

#Строим столбчатую диаграмму важности признаков
plt.figure()
plt.title("Важность признаков")
plt.bar(range(X.shape[1]), importances[indices], color="g", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()